# Аналіз впливу у лінійній регресії

## Yuriy Ryzhkin

### Варіант 4

In [17]:
# read file names from folder 
folder <- paste(getwd(),'/data',sep = '')
filenames=list.files(path = folder,full.names=TRUE)

In [69]:
#create list with dataframes from /data folder
datalist = lapply(filenames,
                  function (x){
                      a <- unlist(strsplit(x,'[_.]'));
                      company <- a[length(a)-1];
                      x0<- read.csv(x,header = F)[,c(1,6)];# using ol
                      colnames(x0)<-c('date',company);
                      x0;
                  })
#merge tables to one
y<-Reduce(function(x,y) {merge(x,y,by="date")}, datalist)

In [82]:
df<-y[-nrow(y),-1]
df$tomorrow_apd <- y$apd[-1]
head(df)
#columns is closed prices of companies

,amgn,amp,amt,amzn,an,anf,aon,apa,apc,apd,tomorrow_apd
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,76.6332,29.6218,24.1784,45.99,19.86,44.4525,28.5404,70.9727,45.7598,46.0207,46.3674
2,77.1332,29.2406,24.0615,44.90,19.82,43.6285,28.6119,67.7776,43.4416,46.3674,45.0466
3,75.1332,29.0327,23.4377,44.47,19.50,43.2032,28.8173,65.2821,42.0754,45.0466,44.7825
4,72.9025,28.9114,23.3305,44.03,19.30,45.8170,28.3797,62.0681,41.9374,44.7825,45.3108
5,74.0948,29.0240,23.5547,43.66,19.29,46.1359,28.4154,62.8432,42.7276,45.3108,44.8073
6,73.8929,28.6428,23.0772,43.78,19.21,45.2056,27.9508,62.4178,42.4134,44.8073,44.6422


Давайте спробуємо спрогнозувати завтрішню ціну компанії apd, використовуючи ціни 9 інших компаній.

Lets try predict tomorrow price of apd company from other nine companies

In [105]:
number_rows<-nrow(df)
model_other_companies<-lm(tomorrow_apd~.-tomorrow_apd-apd,data=df[(number_rows-50):number_rows,])
summary(model_other_companies)


Call:
lm(formula = tomorrow_apd ~ . - tomorrow_apd - apd, data = df[(number_rows - 
    50):number_rows, ])

Residuals:
    Min      1Q  Median      3Q     Max 
-3.4701 -0.9844 -0.2693  1.2882  6.7459 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 94.76930   28.01091   3.383 0.001586 ** 
amgn         0.50324    0.14089   3.572 0.000923 ***
amp          1.93001    0.44679   4.320 9.68e-05 ***
amt         -0.14082    0.20870  -0.675 0.503606    
amzn        -0.07957    0.04901  -1.623 0.112171    
an          -0.15408    0.41761  -0.369 0.714062    
anf         -0.10216    0.32151  -0.318 0.752286    
aon         -1.36207    0.57489  -2.369 0.022615 *  
apa         -0.15864    0.23441  -0.677 0.502359    
apc         -0.70754    0.31802  -2.225 0.031656 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.926 on 41 degrees of freedom
Multiple R-squared:  0.8955,	Adjusted R-squared:  0.8725 
F-statistic: 39.0

Наше p-value: < 2.2e-16, що говорить нам про те, що наша ціна доволі сильно корелює з вчорашніми цінами авіакомпаній.
Як ми бачимо у нас доволі непогана  метрика Multiple R-squared.
Наша змінна значущо залежить від декількох регресорів , а саме apc,aon,amp,amgn. Рівень alpha беремо, як 0.05


Давайте тепер спробуємо прибрати незначущі регресори і побудувати регресію вже на них

In [109]:
model_signif_companies <-lm(tomorrow_apd~apc+aon+amp+amgn ,data=df[(number_rows-50):number_rows,])
summary(model_signif_companies)


Call:
lm(formula = tomorrow_apd ~ apc + aon + amp + amgn, data = df[(number_rows - 
    50):number_rows, ])

Residuals:
    Min      1Q  Median      3Q     Max 
-3.6081 -1.2058 -0.3661  1.2199  6.5928 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  74.7424    16.3116   4.582 3.52e-05 ***
apc          -1.0057     0.2226  -4.517 4.35e-05 ***
aon          -1.3375     0.5165  -2.590   0.0128 *  
amp           1.7313     0.2654   6.524 4.73e-08 ***
amgn          0.5304     0.1177   4.508 4.49e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.925 on 46 degrees of freedom
Multiple R-squared:  0.8829,	Adjusted R-squared:  0.8728 
F-statistic: 86.75 on 4 and 46 DF,  p-value: < 2.2e-16


Multiple R-squared ненабагато зменшився, що говорить, що в відкинутих регресорах було мало даних.

Спробуємо тепер використати в прогнозуванні вчорашню ціну нашої компанії

In [110]:
model_all <-lm(tomorrow_apd~.-tomorrow_apd,data=df[(number_rows-50):number_rows,])
summary(model_all)


Call:
lm(formula = tomorrow_apd ~ . - tomorrow_apd, data = df[(number_rows - 
    50):number_rows, ])

Residuals:
    Min      1Q  Median      3Q     Max 
-2.4658 -0.7668 -0.1917  0.6829  6.1666 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) 46.32546   25.25717   1.834   0.0741 .  
amgn         0.20742    0.13219   1.569   0.1245    
amp          0.45083    0.48760   0.925   0.3607    
amt         -0.15192    0.17100  -0.888   0.3796    
amzn        -0.02942    0.04161  -0.707   0.4836    
an          -0.30113    0.34363  -0.876   0.3861    
anf          0.20647    0.27184   0.760   0.4520    
aon         -0.51409    0.50590  -1.016   0.3156    
apa         -0.21741    0.19247  -1.130   0.2654    
apc         -0.02170    0.30032  -0.072   0.9428    
apd          0.71306    0.15529   4.592  4.3e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.578 on 40 degrees of freedom
Multiple R-squared:  0.9316,

Показник Multiple R-squared збільшився, але при цьому знащучим в прогнозуванні виявився лише один регресор, а саме apd,або сьогоднішня ціна на нашу компанію.

Спробуємо тепер використати лише вчорашню ціну

In [111]:
model_all <-lm(tomorrow_apd~apd,data=df[(number_rows-50):number_rows,])
summary(model_all)


Call:
lm(formula = tomorrow_apd ~ apd, data = df[(number_rows - 50):number_rows, 
    ])

Residuals:
    Min      1Q  Median      3Q     Max 
-3.0942 -0.5212 -0.0511  0.3741  6.3387 

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.99613    4.11481   0.485     0.63    
apd          0.98205    0.04219  23.276   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 1.57 on 49 degrees of freedom
Multiple R-squared:  0.9171,	Adjusted R-squared:  0.9154 
F-statistic: 541.8 on 1 and 49 DF,  p-value: < 2.2e-16


Наше припущення справдилось. Дійсно, наша завтрішня ціна доволі сильно корелює з нашою сьогоднішньою. При цьому судячи по відносно невеликому зміненні Multiple R-squared інші регресори все таки не сильно впливають на нашу модель.